In [14]:
import pandas as pd
import os

In [15]:
# directory = '/data/shared/mtsd_v2_fully_annotated/cropped_signs_v6/train'
# print(os.listdir(directory))
# for label in range(0, 12):
#     count = 0
#     for filename in os.listdir(os.path.join(directory, str(label))):
#         if filename.endswith(".jpg") or filename.endswith(".png"):
#             count += 1
#     print(label, count)


In [16]:
#mapillary vistas 

# model 1
# mv_df = pd.read_csv('../mapillaryvistas_model_3_no_background.csv')
# mv_df = pd.read_csv('../mapillaryvistas_model_3_no_background__.csv')
# mv_df = pd.read_csv('../mapillaryvistas_model_3.csv')
mv_df = pd.read_csv('../mapillaryvistas_model_3_updated.csv')

# model 2
# mv_df = pd.read_csv('../mapillaryvistas_model_2.csv')

# mv_df = pd.read_csv('../mapillaryvistas_model_2_no_background.csv')

# only circles and triangles on rects
# mv_df = pd.read_csv('../mapillaryvistas_model_2_no_background_strict.csv')

# any shape except diamond on rects
# mv_df = pd.read_csv('../mapillaryvistas_model_2_no_background_strict_diamond.csv')

mv_df

,filename,object_id,shape,predicted_shape,predicted_class,group,batch_number,row,column
0,--NSVcUgfVhFd6uzkqHOOg_48.png,48,diamond,diamond,diamond-600.0,1,0,0,a
1,--NSVcUgfVhFd6uzkqHOOg_49.png,49,rect,other,other-0.0-0.0,3,0,0,a
2,--WOpVBuHlCygAUADkttpg_56.png,56,rect,other,other-0.0-0.0,3,0,0,b
3,--WOpVBuHlCygAUADkttpg_66.png,66,rect,square,square-600.0,1,0,0,a
4,--WOpVBuHlCygAUADkttpg_70.png,70,circle,circle,circle-750.0,1,0,0,a
...,...,...,...,...,...,...,...,...,...
27995,z0MyWVxmddcDQdyTnuUz0Q_71.png,71,rect,other,other-0.0-0.0,3,-112,19,d
27996,z0S45crPTyXsXbbsBHDAIQ_14.png,14,circle,circle,circle-750.0,1,49,17,a
27997,z0duqxzEdnmI9fFbk0oxCg_55.png,55,circle,circle,circle-750.0,1,49,17,b
27998,z0vX2_oEI87MXPrVMgIkHw_8.png,8,rect,other,other-0.0-0.0,3,-112,19,e


In [17]:
mv_df['group'].value_counts()

3    16551
1    10174
2     1275
Name: group, dtype: int64

In [18]:
mv_df['group'].value_counts()/len(mv_df)

3    0.591107
1    0.363357
2    0.045536
Name: group, dtype: float64

In [19]:
grouped_df = mv_df.groupby(by=['shape', 'predicted_shape', 'group']).count()
grouped_df = grouped_df.reset_index()
grouped_df[grouped_df['group'] == 1][['shape', 'predicted_shape', 'group', 'filename']].rename({'filename':'count'}, axis=1)

,shape,predicted_shape,group,count
0,circle,circle,1,4987
9,diamond,diamond,1,1229
18,octagon,octagon,1,3
30,pentagon,pentagon,1,81
35,rect,circle,1,889
36,rect,diamond,1,23
37,rect,octagon,1,20
39,rect,pentagon,1,3
40,rect,rect,1,1145
41,rect,square,1,654


In [20]:
grouped_df = mv_df.groupby(by=['shape', 'predicted_shape', 'group']).count()
grouped_df = grouped_df.reset_index()
grouped_df[grouped_df['group'] == 2][['shape', 'predicted_shape', 'group', 'filename']].rename({'filename':'count'}, axis=1)

,shape,predicted_shape,group,count
1,circle,diamond,2,9
2,circle,octagon,2,502
3,circle,pentagon,2,1
4,circle,rect,2,17
5,circle,square,2,229
6,circle,triangle,2,2
7,circle,triangle_inverted,2,8
8,diamond,circle,2,17
10,diamond,octagon,2,1
12,diamond,pentagon,2,1


In [21]:
grouped_df = mv_df.groupby(by=['shape', 'predicted_shape', 'group']).count()
grouped_df = grouped_df.reset_index()
grouped_df[grouped_df['group'] == 3][['shape', 'predicted_shape', 'group', 'filename']].rename({'filename':'count'}, axis=1)

,shape,predicted_shape,group,count
11,diamond,other,3,429
19,octagon,other,3,55
23,other,other,3,582
29,pentagon,other,3,739
38,rect,other,3,14500
47,triangle,other,3,148
51,triangle_inverted,other,3,98


In [22]:
# contour detector
mv_df['shape'].value_counts()

rect                 17357
circle                5755
diamond               1704
pentagon              1085
other                  759
triangle_inverted      702
triangle               576
octagon                 62
Name: shape, dtype: int64

In [23]:
# resnet
mv_df['predicted_class'].value_counts().sort_values(ascending=False)

other-0.0-0.0               16551
circle-750.0                 6165
diamond-915.0                1143
square-600.0                  890
triangle_inverted-1220.0      707
octagon-915.0                 567
rect-458.0-610.0              487
triangle-900.0                479
rect-762.0-915.0              468
rect-915.0-1220.0             233
diamond-600.0                 224
pentagon-915.0                 86
Name: predicted_class, dtype: int64